In [239]:
import numpy as np
import pandas as pd
import glob
import os
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Merge historical data

data can be found on [ICPSR website](https://www.icpsr.umich.edu/web/ICPSR/series/64/studies)

In [240]:
%%time

dfs = []
files = glob.glob('../../data/*.tsv')
# fields = ['NEWRACE', 'AGE', 'IRSEX', 'HERAGE', 'LSDAGE', 'PCPAGE',
#           'CRKAGE', 'ECSAGE', 'COCAGE', 'METHAGE', 'CIGAGE', 'SNUFTRY',
#           'CHEWTRY', 'MTHAAGE', 'OXYCAGE', 'CIGTRY ', 'SEDAGE', 'STIMAGE',
#           'TRANAGE', 'CIGARTRY', 'INHAGE', 'MJAGE', 'ANALAGE', 'BLNTAGE', 'ALCTRY', 'USEACM'] 

general = ['NEWRACE', 'AGE', 'IRSEX', 'USEACM']
cig = ['CIGTRY']
alc = ['ALCTRY']
mar = ['MJAGE']
other_tobacco = ['CIGARTRY', 'CHEWTRY', 'SNUFTRY', 'SLTTRY']
other_drugs = ['COCAGE', 'HALLAGE', 'HERAGE', 'INHAGE', 'ANALAGE', 'SEDAGE', 'STIMAGE', 'TRANAGE']
fields = general + cig + alc + mar + other_tobacco + other_drugs

for file in files:
    df_n1 = pd.read_csv(file, sep='\t', skipinitialspace=True, nrows=1)
    current_columns = []
    for field in fields:
        field = field.strip()
        if field in df_n1.columns:
            current_columns.append(field)
        elif f'{field}2' in df_n1.columns:
            current_columns.append(f'{field}2')
        else:
            print(f'field {field} not in {file}')
    current_columns += ['CIGTRY']
    df = pd.read_csv(file, sep='\t', skipinitialspace=True, usecols=current_columns)
    df['file_name'] = os.path.basename(file)
    dfs.append(df)

CPU times: user 16.9 s, sys: 688 ms, total: 17.6 s
Wall time: 17.6 s


In [279]:
classes

['CIGTRY',
 'ALCTRY',
 'MJAGE',
 'CIGARTRY',
 'CHEWTRY',
 'SNUFTRY',
 'SLTTRY',
 'COCAGE',
 'HALLAGE',
 'HERAGE',
 'INHAGE',
 'ANALAGE',
 'SEDAGE',
 'STIMAGE',
 'TRANAGE']

In [241]:
main_df = pd.concat(dfs)
# main_df_backlog = main_df.copy()
main_df.shape  # data diff 613774 (real) - 615701 (paper) = -1927 difference

(613974, 20)

In [242]:
%%time
np.random.seed(42)
main_df = pd.concat(dfs)
file_to_year_mapping = {f:2004+i for i, f in enumerate(sorted([os.path.basename(f) for f in files]))}
class_mapping = {
    'MARIJUANA': 'MARIJUANA',
    'MJAGE': 'MARIJUANA',
    'ALCOHOL': 'ALCOHOL',
    'ALCTRY': 'ALCOHOL',
    'CIGARETTES': 'CIGARETTES',
    'CIGTRY': 'CIGARETTES',
    'NO_DRUG_USE': 'NO_DRUG_USE',
    'NOUSAGE': 'NO_DRUG_USE',
    'OTHER_TABACCO': 'OTHER_TABACCO',
    'CIGARTRY': 'OTHER_TABACCO',
    'SNUFTRY': 'OTHER_TABACCO',
    'CHEWTRY': 'OTHER_TABACCO',
    'SLTTRY': 'OTHER_TABACCO',
    'OTHER_DRUGS': 'OTHER_DRUGS',
    'HERAGE': 'OTHER_DRUGS',
    'COCAGE': 'OTHER_DRUGS',
    'SEDAGE': 'OTHER_DRUGS',
    'STIMAGE': 'OTHER_DRUGS',
    'TRANAGE': 'OTHER_DRUGS',
    'INHAGE': 'OTHER_DRUGS',
    'ANALAGE': 'OTHER_DRUGS',
    'HALLAGE': 'OTHER_DRUGS'
}
age_to_group_mapping = {
    12: '12-13',
    13: '12-13',
    14: '14-15',
    15: '14-15',
    16: '16-17',
    17: '16-17',
    18: '18-19',
    19: '18-19',
    20: '20-21',
    21: '20-21'
}
race_mapping = {1: 'White', 2: 'Black', 3: 'AI/AN', 4: 'NHOPI', 5: 'Asian', 6: 'Multi-racial', 7: 'Hispanic'}
use_acm_map = {
    1: 'ALCOHOL',
    2: 'CIGARETTES',
    3: 'MARIJUANA',
    4: 'ALCOHOL',
    5: 'CIGARETTES',
    6: 'MARIJUANA',
#     11: 'ALCTRY',
#     12: 'CIGTRY',
#     13: 'MJAGE',
    91: 'NO_DRUG_USE',
}
classes = cig + alc + mar + other_tobacco + other_drugs
main_df = main_df[(main_df['AGE2'] < 11)]  # filter people < 22 yo
main_df['MINAGE'] = main_df[classes].values.min(axis=1)
main_df['MINAGE'] = np.where(main_df['MINAGE'] > 900, 999, main_df['MINAGE'])
main_df['MINAGE_CLASS'] = np.where(main_df['MINAGE'] > 900, 'NO_DRUG_USE', None)
main_df['CLASSES_LIST'] = np.where(main_df['MINAGE'] > 900, 'NO_DRUG_USE', None)
main_df = main_df[~(main_df.MINAGE_CLASS=='NO_DRUG_USE')==(main_df.USEACM==99)] # remove where unknown class
main_df['YEAR'] = main_df['file_name'].map(file_to_year_mapping) # infer year
main_df['SEX'] = main_df['IRSEX'].map({1: 'Male', 2: 'Female'})
main_df['AGE'] = main_df['AGE2'].map({i:i+11 for i in range(1, 11)})
main_df['RACE'] = main_df['NEWRACE2'].map(race_mapping)
main_df['AGE_GROUP'] = main_df['AGE'].map(age_to_group_mapping)
main_df.reset_index(inplace=True, drop=True)
main_df.head()

CPU times: user 93.6 ms, sys: 32 ms, total: 126 ms
Wall time: 125 ms


,CIGTRY,SNUFTRY,CHEWTRY,SLTTRY,CIGARTRY,ALCTRY,MJAGE,COCAGE,HERAGE,HALLAGE,INHAGE,ANALAGE,TRANAGE,STIMAGE,SEDAGE,USEACM,AGE2,IRSEX,NEWRACE2,file_name,MINAGE,MINAGE_CLASS,CLASSES_LIST,YEAR,SEX,AGE,RACE,AGE_GROUP
0,991,991,991,991,991,14,14,991,991,991,991,991,991,991,991,99,4,2,7,32722-0001-Data.tsv,14,None,None,2010,Female,15,Hispanic,14-15
1,14,991,991,991,991,13,991,991,991,991,991,991,991,991,991,99,7,2,1,32722-0001-Data.tsv,13,None,None,2010,Female,18,White,18-19
2,991,991,991,991,991,991,991,991,991,991,991,991,991,991,991,91,2,1,7,32722-0001-Data.tsv,999,NO_DRUG_USE,NO_DRUG_USE,2010,Male,13,Hispanic,12-13
3,15,991,991,991,15,16,16,18,991,991,991,18,991,991,991,99,9,1,7,32722-0001-Data.tsv,15,None,None,2010,Male,20,Hispanic,20-21
4,991,991,991,991,17,17,18,991,991,991,991,991,991,991,991,99,9,1,1,32722-0001-Data.tsv,17,None,None,2010,Male,20,White,20-21


In [243]:
main_df.shape

(277254, 28)

In [244]:
main_df.MINAGE_CLASS.value_counts()/main_df.shape[0]

NO_DRUG_USE    0.424557
Name: MINAGE_CLASS, dtype: float64

In [245]:
main_df.SEX.value_counts()

Male      138850
Female    138404
Name: SEX, dtype: int64

In [246]:
main_df.AGE_GROUP.value_counts().sort_index()

12-13    57906
14-15    60874
16-17    62424
18-19    49444
20-21    46606
Name: AGE_GROUP, dtype: int64

In [247]:
main_df.RACE.value_counts()

White           164282
Hispanic         49292
Black            37656
Multi-racial     11080
Asian             9434
AI/AN             4210
NHOPI             1300
Name: RACE, dtype: int64

In [248]:
main_df.YEAR.value_counts().sort_index()

2004    26046
2005    26307
2006    25704
2007    25513
2008    25633
2009    25462
2010    26375
2011    27249
2012    25045
2013    25024
2014    18896
Name: YEAR, dtype: int64

In [249]:
main_df.MINAGE_CLASS.value_counts()

NO_DRUG_USE    117710
Name: MINAGE_CLASS, dtype: int64

In [250]:
main_df.reset_index(inplace=True, drop=True)
main_df.head()

,CIGTRY,SNUFTRY,CHEWTRY,SLTTRY,CIGARTRY,ALCTRY,MJAGE,COCAGE,HERAGE,HALLAGE,INHAGE,ANALAGE,TRANAGE,STIMAGE,SEDAGE,USEACM,AGE2,IRSEX,NEWRACE2,file_name,MINAGE,MINAGE_CLASS,CLASSES_LIST,YEAR,SEX,AGE,RACE,AGE_GROUP
0,991,991,991,991,991,14,14,991,991,991,991,991,991,991,991,99,4,2,7,32722-0001-Data.tsv,14,None,None,2010,Female,15,Hispanic,14-15
1,14,991,991,991,991,13,991,991,991,991,991,991,991,991,991,99,7,2,1,32722-0001-Data.tsv,13,None,None,2010,Female,18,White,18-19
2,991,991,991,991,991,991,991,991,991,991,991,991,991,991,991,91,2,1,7,32722-0001-Data.tsv,999,NO_DRUG_USE,NO_DRUG_USE,2010,Male,13,Hispanic,12-13
3,15,991,991,991,15,16,16,18,991,991,991,18,991,991,991,99,9,1,7,32722-0001-Data.tsv,15,None,None,2010,Male,20,Hispanic,20-21
4,991,991,991,991,17,17,18,991,991,991,991,991,991,991,991,99,9,1,1,32722-0001-Data.tsv,17,None,None,2010,Male,20,White,20-21


In [251]:
%%time
for i, row in main_df.iterrows():
#     if i > 2000:
#         break
    if row['MINAGE'] > 900:
        continue
    several_substances = sorted(row[classes][row[classes].apply(lambda x: x==row['MINAGE'])].index.values)
    several_substances_mapped = sorted(list(set([class_mapping[s] for s in several_substances])))
    main_df.at[i, 'CLASSES_LIST'] = '/'.join(several_substances_mapped)
    if len(several_substances_mapped) == 1:
        main_df.at[i, 'MINAGE_CLASS'] = several_substances_mapped[0]  
    else:
        main_df.at[i, 'MINAGE_CLASS'] = use_acm_map.get(row['USEACM']) or np.random.choice(several_substances_mapped)
main_df.head()

CPU times: user 1min 28s, sys: 260 ms, total: 1min 28s
Wall time: 1min 28s


,CIGTRY,SNUFTRY,CHEWTRY,SLTTRY,CIGARTRY,ALCTRY,MJAGE,COCAGE,HERAGE,HALLAGE,INHAGE,ANALAGE,TRANAGE,STIMAGE,SEDAGE,USEACM,AGE2,IRSEX,NEWRACE2,file_name,MINAGE,MINAGE_CLASS,CLASSES_LIST,YEAR,SEX,AGE,RACE,AGE_GROUP
0,991,991,991,991,991,14,14,991,991,991,991,991,991,991,991,99,4,2,7,32722-0001-Data.tsv,14,ALCOHOL,ALCOHOL/MARIJUANA,2010,Female,15,Hispanic,14-15
1,14,991,991,991,991,13,991,991,991,991,991,991,991,991,991,99,7,2,1,32722-0001-Data.tsv,13,ALCOHOL,ALCOHOL,2010,Female,18,White,18-19
2,991,991,991,991,991,991,991,991,991,991,991,991,991,991,991,91,2,1,7,32722-0001-Data.tsv,999,NO_DRUG_USE,NO_DRUG_USE,2010,Male,13,Hispanic,12-13
3,15,991,991,991,15,16,16,18,991,991,991,18,991,991,991,99,9,1,7,32722-0001-Data.tsv,15,OTHER_TABACCO,CIGARETTES/OTHER_TABACCO,2010,Male,20,Hispanic,20-21
4,991,991,991,991,17,17,18,991,991,991,991,991,991,991,991,99,9,1,1,32722-0001-Data.tsv,17,ALCOHOL,ALCOHOL/OTHER_TABACCO,2010,Male,20,White,20-21


In [252]:
main_df['CLASS'] = main_df['MINAGE_CLASS'].map(class_mapping)
main_df.reset_index(inplace=True, drop=True)
# main_df.to_csv('fairman19marijuana_dataframe.csv')

In [253]:
main_df['CLASS'].value_counts()

NO_DRUG_USE      117710
ALCOHOL           79901
CIGARETTES        40635
MARIJUANA         17969
OTHER_TABACCO     11302
OTHER_DRUGS        9737
Name: CLASS, dtype: int64

In [289]:
main_df[['RACE', 'CLASS']].value_counts() * 100/main_df[['RACE']].value_counts()

RACE          CLASS        
AI/AN         NO_DRUG_USE      33.467933
              CIGARETTES       23.895487
              ALCOHOL          17.719715
              MARIJUANA        12.066508
              OTHER_TABACCO     7.743468
              OTHER_DRUGS       5.106888
Asian         NO_DRUG_USE      54.600382
              ALCOHOL          28.323087
              CIGARETTES        9.847361
              MARIJUANA         3.264787
              OTHER_DRUGS       2.713589
              OTHER_TABACCO     1.250795
Black         NO_DRUG_USE      45.955492
              ALCOHOL          25.748885
              CIGARETTES       11.982154
              MARIJUANA         9.971850
              OTHER_DRUGS       3.866582
              OTHER_TABACCO     2.475037
Hispanic      NO_DRUG_USE      43.211880
              ALCOHOL          29.692445
              CIGARETTES       13.117747
              MARIJUANA         7.463686
              OTHER_DRUGS       4.016879
              OTHER_TABACCO  

In [286]:
main_df[['SEX']].value_counts()

CLASS        
NO_DRUG_USE      117710
ALCOHOL           79901
CIGARETTES        40635
MARIJUANA         17969
OTHER_TABACCO     11302
OTHER_DRUGS        9737
dtype: int64

In [261]:
main_df.CLASSES_LIST.value_counts()

NO_DRUG_USE                                       117710
ALCOHOL                                            68318
CIGARETTES                                         30879
MARIJUANA                                          10890
ALCOHOL/CIGARETTES                                  8307
OTHER_DRUGS                                         7132
ALCOHOL/MARIJUANA                                   6518
OTHER_TABACCO                                       6098
CIGARETTES/MARIJUANA                                4102
ALCOHOL/OTHER_TABACCO                               3384
CIGARETTES/OTHER_TABACCO                            3213
ALCOHOL/OTHER_DRUGS                                 2298
ALCOHOL/CIGARETTES/OTHER_TABACCO                    1860
ALCOHOL/MARIJUANA/OTHER_TABACCO                     1033
CIGARETTES/MARIJUANA/OTHER_TABACCO                   885
CIGARETTES/OTHER_DRUGS                               885
MARIJUANA/OTHER_TABACCO                              799
ALCOHOL/MARIJUANA/OTHER_DRUGS  

In [262]:
main_df[main_df.CLASS.isna()].head()

,CIGTRY,SNUFTRY,CHEWTRY,SLTTRY,CIGARTRY,ALCTRY,MJAGE,COCAGE,HERAGE,HALLAGE,INHAGE,ANALAGE,TRANAGE,STIMAGE,SEDAGE,USEACM,AGE2,IRSEX,NEWRACE2,file_name,MINAGE,MINAGE_CLASS,CLASSES_LIST,YEAR,SEX,AGE,RACE,AGE_GROUP,CLASS


In [263]:
# 275,559
df = main_df[['YEAR', 'CLASS', 'SEX', 'RACE', 'AGE_GROUP', 'AGE', 'MINAGE']]
df['SEX'] = df['SEX'].astype('category')
df['RACE'] = df['RACE'].astype('category')
df['AGE_GROUP'] = df['AGE_GROUP'].astype('category')
df['CLASS'] = df['CLASS'].astype('category')
df['YEAR'] = df['YEAR'].astype('category')
df['AGE'] = df['AGE'].astype(np.int32)
df['MINAGE'] = df['MINAGE'].astype(np.int32)
df.shape

(277254, 7)

In [264]:
275559 - main_df.shape[0]

-1695

In [266]:
import pickle

with open('../fairman19marijuana_dataframe.pickle', 'wb') as pickle_file:
    pickle.dump(df, pickle_file)

# check

In [267]:
df = pd.read_pickle('../fairman19marijuana_dataframe.pickle')
df.head()

,YEAR,CLASS,SEX,RACE,AGE_GROUP,AGE,MINAGE
0,2010,ALCOHOL,Female,Hispanic,14-15,15,14
1,2010,ALCOHOL,Female,White,18-19,18,13
2,2010,NO_DRUG_USE,Male,Hispanic,12-13,13,999
3,2010,OTHER_TABACCO,Male,Hispanic,20-21,20,15
4,2010,ALCOHOL,Male,White,20-21,20,17


In [268]:
df.CLASS.value_counts()

NO_DRUG_USE      117710
ALCOHOL           79901
CIGARETTES        40635
MARIJUANA         17969
OTHER_TABACCO     11302
OTHER_DRUGS        9737
Name: CLASS, dtype: int64

In [273]:
text = """
No drug use(n=116,320) - 
Cigarettes(n=39,735)
Alcohol(n=77,830)
Marijuana(n=15,267)
Other Tobaccoa(n=9,798)
Other Drugsb(n=16,609)
aOther tobacco included cigars CIGARTRY, pipes NONE, chew CHEWTRY, snuff SNUFTRY, and smokeless tobacco SLTTRY.
bOther drugs included non-medical use of cocaine COCAGE, hallucinogens HALLAGE, heroin HERAGE, inhalants INHAGE, analgesics ANALAGE, sedatives SEDAGE, stimulants STIMAGE, and tranquillizers TRANAGE.
"""
text

'\nNo drug use(n=116,320) - \nCigarettes(n=39,735)\nAlcohol(n=77,830)\nMarijuana(n=15,267)\nOther Tobaccoa(n=9,798)\nOther Drugsb(n=16,609)\naOther tobacco included cigars CIGARTRY, pipes NONE, chew CHEWTRY, snuff SNUFTRY, and smokeless tobacco SLTTRY.\nbOther drugs included non-medical use of cocaine COCAGE, hallucinogens HALLAGE, heroin HERAGE, inhalants INHAGE, analgesics ANALAGE, sedatives SEDAGE, stimulants STIMAGE, and tranquillizers TRANAGE.\n'

In [275]:
paper_counts = [116320, 77830, 39735, 16609, 9798, 15267]
for true, (name, estimated) in zip(paper_counts, df.CLASS.value_counts().items()):
    print(name, true, estimated, true - estimated, 'MORE' if estimated > true else 'LESS') 

NO_DRUG_USE 116320 117710 -1390 MORE
ALCOHOL 77830 79901 -2071 MORE
CIGARETTES 39735 40635 -900 MORE
MARIJUANA 16609 17969 -1360 MORE
OTHER_TABACCO 9798 11302 -1504 MORE
OTHER_DRUGS 15267 9737 5530 LESS


In [277]:
mean_age_first_use_2004 = df[
    (df.CLASS == 'MARIJUANA') & (df.YEAR == 2004)]['MINAGE'].mean()
mean_age_first_use_2014 = df[
    (df.CLASS == 'MARIJUANA') & (df.YEAR == 2014)]['MINAGE'].mean()
mean_age_first_use_2004, mean_age_first_use_2014

(13.511398176291793, 14.089460784313726)

# temp

In [ ]:
df[(df.CLASS == 'MARIJUANA') & (df.YEAR == 2004)].shape[0]*100/df[(df.YEAR == 2004)].shape[0]

In [ ]:
df[(df.CLASS == 'MARIJUANA') & (df.YEAR == 2014)].shape[0]*100/df[(df.YEAR == 2014)].shape[0]

In [ ]:
df[(df.CLASS == 'MARIJUANA')].shape[0]*100/df.shape[0]
# df[(df.CLASS == 'ALCOHOL')].shape[0]*100/df.shape[0]
# df[(df.CLASS == 'CIGARETTES')].shape[0]*100/df.shape[0]
# main_df[(main_df.CLASS == 'OTHER_DRUGS')].shape[0]*100/main_df.shape[0]
# main_df[(main_df.CLASS == 'OTHER_TABACCO')].shape[0]*100/main_df.shape[0]

In [ ]:
main_df[(main_df.CLASS == 'MARIJUANA')].shape[0]*100/main_df.shape[0] # 5.8% <
# main_df[(main_df.CLASS == 'ALCOHOL')].shape[0]*100/main_df.shape[0] # 29.8% for alcohol <
# main_df[(main_df.CLASS == 'CIGARETTES')].shape[0]*100/main_df.shape[0] # 14.2% for cigarettes ~>
# main_df[(main_df.CLASS == 'OTHER_DRUGS')].shape[0]*100/main_df.shape[0] # 3.6% for other tobacco >
# main_df[(main_df.CLASS == 'OTHER_TABACCO')].shape[0]*100/main_df.shape[0] # 5.9% other drugs <